##Project for a B2B professional services company

## Task 1
#### Has user activity or engagement dropped, increased or remained stable? What is the extent of change in user activity/engagement?

##### First let us check the duration of the dataset

In [0]:
%sql 
SELECT min(occurred_at), max(occurred_at) FROM events

min(occurred_at),max(occurred_at)
2014-05-01 00:54:09,2014-08-31 20:40:32


##### Since the data is for four months (from May to August), let us plot total user engagement per day, week and month for the entire dataset

In [0]:
%sql
-- by day
SELECT
   to_date(occurred_at) as date,
   COUNT (*) as total_engagement 
FROM
   events
WHERE
   event_type = 'engagement' 
GROUP BY
   to_date(occurred_at) 
ORDER BY
   to_date(occurred_at)

date,total_engagement
2014-05-01,1503
2014-05-02,1969
2014-05-03,442
2014-05-04,271
2014-05-05,1200
2014-05-06,1494
2014-05-07,1361
2014-05-08,1496
2014-05-09,1468
2014-05-10,449


In [0]:
%sql
-- by week
SELECT
   EXTRACT(week from occurred_at) as week,
   COUNT (*) as total_engagement 
FROM
   events
WHERE
   event_type = 'engagement' 
GROUP BY
   week 
ORDER BY
   week

week,total_engagement
18,4185
19,7787
20,7979
21,7831
22,7590
23,8133
24,7518
25,7958
26,7639
27,8003


In [0]:
%sql
-- by month
SELECT
   EXTRACT(month from occurred_at) as month,
   COUNT (*) as total_engagement 
FROM
   events 
WHERE
   event_type = 'engagement' 
GROUP BY
   month 
ORDER BY
   month

month,total_engagement
5,35078
6,32764
7,36055
8,26846


#### Few Initial Observations
  - We can see that there is a decline is user engagement over the weekends, which is expected because very few people would be working over the weekend
  - We also see a visible decline in user engagement starting from August

#### Given the above obervations, let us analyze the engagement difference week over week and month over month

In [0]:
%sql
-- Analyzing engagement difference by week
WITH weekly_engagement AS 
(
   SELECT
      EXTRACT(week from occurred_at) AS week,
      COUNT (*) as total_engagement 
   FROM
      events
   WHERE
      event_type = 'engagement' 
     AND occurred_at > '2014-05-05' 
   GROUP BY
      1 
   ORDER BY
      1
),
engagement_previous AS 
(
   SELECT
      week,
      total_engagement,
      LAG(total_engagement, 1) OVER ( 
   ORDER BY
      week ) previous_week_engagement 
   FROM
      weekly_engagement 
)
SELECT
   week,
   total_engagement,
   CASE WHEN previous_week_engagement IS NULL THEN 0 ELSE 100*(total_engagement - previous_week_engagement) / previous_week_engagement END as engagement_difference 
FROM
   engagement_previous

week,total_engagement,engagement_difference
19,7787,0.0
20,7979,2.4656478746628996
21,7831,-1.8548690312069183
22,7590,-3.077512450517175
23,8133,7.154150197628459
24,7518,-7.561785319070454
25,7958,5.852620377760043
26,7639,-4.008544860517718
27,8003,4.765021599685823
28,7779,-2.798950393602399


- We can see that the engagement dropped by almost 22% week over week in the first week of August which seems like an anomaly compared to the rest of the weeks

In [0]:
%sql
-- Analyzing engagement difference month over month
WITH monthly_engagement AS 
(
   SELECT
      EXTRACT(month from occurred_at) AS month,
      COUNT (*) as total_engagement 
   FROM
      events 
   WHERE
      event_type = 'engagement'  
   GROUP BY
      1 
   ORDER BY
      1
),
engagement_previous AS 
(
   SELECT
      month,
      total_engagement,
      LAG(total_engagement, 1) OVER ( 
   ORDER BY
      month ) previous_month_engagement 
   FROM
      monthly_engagement 
)
SELECT
   month,
   total_engagement,
   CASE WHEN previous_month_engagement IS NULL THEN 0 ELSE 100*(total_engagement - previous_month_engagement) / previous_month_engagement END as engagement_difference 
FROM
   engagement_previous

month,total_engagement,engagement_difference
5,35078,0.0
6,32764,-6.5967272934602885
7,36055,10.044561103650349
8,26846,-25.541533767854666


#### Final Findings

- User activity saw a sharp decline in the month of August (by almost 25%)
- User activity remained almost stable for first three months with a slight drop in the second month
- The extent of change was normal for the first three months while for last two months the extent of change was quite sharp

## Task 2
#### Are there any changes in the three stages of the emails funnel? What changes have you discovered? (Please search online for terms such as sales funnel, marketing funnel to understand the funnel concept).

#### ANSWER FOR TASK 2
##### First let us plot the count of each email action

In [0]:
%sql
SELECT
   action,
   COUNT(*) as num_events 
FROM
   emails 
GROUP BY
   action 
ORDER BY
   num_events DESC

action,num_events
sent_weekly_digest,57267
email_open,20459
email_clickthrough,9010
sent_reengagement_email,3653


##### Now, let us analyze the drop in user actions for each funnel

In [0]:
%sql
WITH email_actions AS 
(
   SELECT
      action,
      COUNT(*) as num_events 
   FROM
      emails 
   GROUP BY
      action 
   ORDER BY
      num_events DESC 
)
,
funnel AS 
(
   SELECT
      action,
      num_events,
      LAG(num_events, 1) OVER ( 
   ORDER BY
      num_events DESC) previous_action_events 
   FROM
      email_actions 
)
SELECT
   action,
   num_events,
   100*num_events / (
   SELECT
      MAX(num_events) 
   FROM
      funnel) AS overall_conversion,
      100*(num_events) / previous_action_events AS stage_conversion 
   FROM
      funnel

action,num_events,overall_conversion,stage_conversion
sent_weekly_digest,57267,100.0,null
email_open,20459,35.72563605566906,35.72563605566906
email_clickthrough,9010,15.7333193636824,44.03929810841195
sent_reengagement_email,3653,6.378891857439712,40.54384017758046


#### Findings
- The percentage change in the change in the email funnel is seen in the table above
- The overall_conversation shows the percentage change with respect to the sent_weekly_digest while the stage_conversation shows the percentage change with respect to the previous stage of the funnel
- From the data we can say that if the email is sent to 100 people then around 35% of them opens the email while 15% of the people clicks on the links followed by 6% of the users sent the reengagement email. 
- So yes there is lot of changes in the three stages of the funnel

## Task 3

#### Are changes in user activity/engagement associated with specific devices or hardware ( mobile phone, tablet computers, desktop computers, etc.)  that customers use to read emails and interact with Zilto’sweb portal?

In [0]:
%sql
WITH emails AS
(
   SELECT
      *,
      to_date(occurred_at) AS date,
      EXTRACT(month FROM occurred_at) AS month 
   FROM
      emails emails
)
,
events AS
(
   SELECT
      user_id,
      to_date(occurred_at) AS date,
      device,
      EXTRACT(month FROM occurred_at) AS month 
   FROM
      events
   ORDER BY
      user_id ASC
)
SELECT
   CASE
      WHEN
         device IN 
         (
            'amazon fire phone',
            'nexus 10',
            'iphone 5',
            'nexus 7',
            'iphone 5s',
            'nexus 5',
            'htc one',
            'iphone 4s',
            'samsung galaxy note',
            'nokia lumia 635',
            'samsung galaxy s4'
         )
      THEN
         'Mobile' 
      WHEN
         device IN 
         (
            'ipad mini', 'samsung galaxy tablet', 'kindle fire', 'ipad air'
         )
      THEN
         'Tablet' 
      WHEN
         device IN 
         (
            'dell inspiron notebook', 'macbook pro', 'asus chromebook', 'windows surface', 'macbook air', 'lenovo thinkpad', 'mac mini', 'acer aspire notebook'
         )
      THEN
         'Laptops' 
      WHEN
         device IN 
         (
            'dell inspiron desktop', 'acer aspire desktop', 'hp pavilion desktop'
         )
      THEN
         'Desktops' 
      ELSE
         'Unknown' 
   END
   AS device_type, 
   emails.month, 
   count(emails.user_id) 
FROM
   emails 
   LEFT JOIN
      events 
      ON emails.user_id = events.user_id 
      AND emails.date = events.date 
WHERE
   action = 'email_clickthrough' 
GROUP BY
   1, 2
ORDER BY
   1, 2

device_type,month,count(user_id)
Desktops,5,306
Desktops,6,505
Desktops,7,454
Desktops,8,323
Laptops,5,3587
Laptops,6,3183
Laptops,7,4103
Laptops,8,3002
Mobile,5,2236
Mobile,6,2556


#### Findings

##### Findings : 
- From the graph we can say that the

## Task 4
#### Are changes in user activity/engagement associated with specific countries or regions of the world where Zilto’s  users are located?

#### ANSWER FOR TASK 4
##### First let us analyze the number of countries/location present in the data

In [0]:
%sql
SELECT COUNT(DISTINCT(location)) FROM events

count(DISTINCT location)
47


##### Let us plot the top 20 countries which have highest engagement overall

In [0]:
%sql
SELECT location, COUNT(*) as engagement_count FROM events GROUP BY location ORDER BY engagement_count DESC LIMIT 20

location,engagement_count
United States,42896
Japan,11729
Germany,10286
United Kingdom,6796
France,6783
Russia,6348
Italy,4970
Brazil,4555
India,4108
Canada,3919


### Final Findings
- From the bar graph we can see the top 20 countries where the user engagement is highest
- We can say that the user engagement is hightest in the USA which is more than 40k
- Following by Japan and Germany which contributes only around 1/3rd of the US users
- So yes we can say that the activity/engagement is accociated with the country

In [0]:
%sql
SELECT location, COUNT(*) as engagement_count FROM events GROUP BY location ORDER BY engagement_count ASC LIMIT 27

location,engagement_count
Portugal,265
Pakistan,332
Iraq,375
Argentina,545
Ireland,545
Singapore,570
Hong Kong,572
Thailand,607
Philippines,635
Greece,672


### Findings
- In the above graph we can see the 27 countries where the user engagement was the lowest
- We can say that Portugal has the least user engagement followed by pakistan and Iraq

In [0]:
%sql
SELECT location, EXTRACT(week from occurred_at) AS week, COUNT (*) AS engagement_count FROM events
WHERE event_type = 'engagement' AND occurred_at > '2014-06-30'
GROUP BY 1, 2
ORDER BY engagement_count DESC

location,week,engagement_count
United States,27,2652
United States,31,2527
United States,30,2431
United States,29,2351
United States,28,2135
United States,33,1852
United States,32,1825
United States,34,1549
United States,35,1522
Japan,31,755


##### Findings
- The major impact is the United States where the user engagement is the highest
- Switzerland had increased the no. of user actitivity learnierly from week 27 to 30 and then sudden decline after week 30 was seen which remained almost constant till week 35
- Neatherland and Spain had increased the number of user activities from 60 to 160 
- Austria had a linear decline in the user activity at week 28 from 180 users to 30 users
- Similarly Venezuela had also seen quite decline in the number of count in the user activity
- ALl the outher contries had somewhat up and downs in the activity but overall not caused much impact and remained stable

### Business Insights from the data set. 


- Now that we have analyzed drop of engagement in august but we don't know what was the reason behind this drop
- Drop of the user engagement activity can be due to various reason such as lost of interest of the users, New better subcription avaiable at loewr cost and more benefits
- Changes in the agreement with respect to the subcribers of a particular country

### Additional Task 
#### Do the answers to any of the initial questions lead you to additional questions or possibilities that should be investigated? If so, what are they and how will you address them?
##### ANSWER
- Yes the intial questions lead to additional questions and possibility that should be investigated such as whay few countries had so less user engagement
- Was there a drop in the user engagement during the weekends and rise in the engagement during the week to understand this we can make two graphs and see the user engagement on weekend Vs Weekdays
- We can also investigate in deep why the user activity dropped in the month of August and see what are the most affecting factors associated with this drop 
- We can also investigate the which days are affecting the drop in the user activities in the particular country (Might be because of public holidays)
- We can also investigate further if the user engagement depends on the culture or language of english speaking users in that particular country